![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. You will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [10]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


In [11]:
# Check for Missing Values
print(cc_apps.isna().sum().sort_values())

# One hot encode dataset
cc_apps_encoded = pd.get_dummies(cc_apps,drop_first=True)

# Define X and Y(target variable)
X = cc_apps_encoded.iloc[:,:-1].values
y = cc_apps_encoded.iloc[:,[-1]].values

# Train, test and split
X_train,X_test,y_train,y_test = train_test_split(X, y,
                                                 test_size = 0.3,
                                                random_state=42)
# Scale the Data
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)


# Instantiate Logistic Regression
logreg = LogisticRegression()
logreg.fit(scaled_X_train,y_train)
y_train_pred = logreg.predict(scaled_X_train)

print(confusion_matrix(y_train,y_train_pred))

tol = [0.01,0.001,0.0001]
max_iter = [100,150,200]

param_grid = dict(tol = tol,max_iter = max_iter)

# Apply Grid Search to try all possible combinations
grid_model = GridSearchCV(estimator = logreg, param_grid = param_grid, cv=5)

grid_model_result = grid_model.fit(scaled_X_train, y_train)

best_train_score,best_train_params = grid_model_result.best_score_, grid_model_result.best_params_

print("Best: %f using %s" % (best_train_score, best_train_params))

best_model = grid_model_result.best_estimator_
best_score = best_model.score(scaled_X_test,y_test)

print("Accuracy of logistic regression classifier: ", best_score)



0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64
[[209   1]
 [  1 272]]
Best: 0.826009 using {'max_iter': 100, 'tol': 0.01}
Accuracy of logistic regression classifier:  0.7922705314009661
